In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install CTransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=14f6284a5075506386ed2245e501838db1cd0dffb4

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="gpt2-medium")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
tokenizer=GPT2Tokenizer.from_pretrained("gpt2-medium")

In [ ]:
def generate_continuously(prompt, max_length):
    chunks = []
    current_chunk = ""
    l=len(prompt)
    while len(current_chunk) < max_length:

        response = pipe(prompt,
                         # Length of each chunk
                        do_sample=False, # Use greedy decoding
                        no_repeat_ngram_size=2, # No repetition penalty
                        early_stopping=True, # Stop as soon as a period is generated
                        pad_token_id=model.config.eos_token_id, # Use the EOS token as the pad token
                        )


        current_chunk += response[0]['generated_text'][len(prompt):]
        l=response[0]['generated_text'][len(prompt):]

        chunks.append(l)

        prompt +=response[0]['generated_text'][len(prompt):]

    return chunks

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
chunks = generate_continuously("What is the number one product ever created and why?", 1000)
generated_text = "".join(chunks)
generated_text

KeyboardInterrupt: ignored

In [ ]:
len(generated_text)

1056

In [ ]:
response = pipe('What is the number one product ever created and why?',
                        # Length of each
                max_new_tokens =300,
                min_new_tokens=100,
                        do_sample=False, # Use greedy decoding
                        no_repeat_ngram_size=2, # No repetition penalty
                        num_beams=2,
                        early_stopping=True, # Stop as soon as a period is generated

                        pad_token_id=model.config.eos_token_id, # Use the EOS token as the pad token
                        ) # Get the attention mask for future chunks
response[0]['generated_text']

'What is the number one product ever created and why?\n\nI think the answer is simple. It\'s a product that is so simple that it\'s almost impossible to explain.\n, the first product I created, was a simple calculator. I wanted to create a calculator that was easy to use, but also had a lot of features that made it stand out from other calculators on the market. The first thing I did was to make it as simple as possible, so that the user could just type in a number and it would do the math for them. Then, I added a few more features, such as the ability to add and subtract numbers, and the addition of fractions, which made the calculator even more useful. In the end, it became the most popular calculator ever made.\n\n\nHow did you come up with the idea for the Calculator 2.0? What was the inspiration behind the name? How long did it take you to develop the product? Did you have any prior experience in the field of computer programming?\n\n, a friend of mine, suggested that I try to de

In [ ]:
import re
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def check_if_generated_text_is_complete(generated_text):
    # Check if the generated text ends with a period or question mark
    if not re.search(r'\.\s*$|\?\s*$', generated_text):
        return False

    # Check if the generated text is too short (less than 10 words)
    if len(re.findall(r'\b\w+\b', generated_text)) < 10:
        return False

    # If both checks pass, the generated text is considered a valid completion
    return True
def generate_text(prompt, tokenizer, model, device):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    model.eval()
    with torch.no_grad():
        output = model.generate(inputs,max_length=49, do_sample=False, # Use greedy decoding
                        no_repeat_ngram_size=2, # No repetition penalty
                                num_beams=2,
                        early_stopping=True, # Stop as soon as a period is generated
                        pad_token_id=model.config.eos_token_id)

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output
prompt="What is the number one product ever created and why?"
# ... tokenizer, model, and device setup ...

# Check if the prompt has already been generated
prompt_complete = False
while not prompt_complete:
    # Generate text with the model
    generated_text = generate_text(prompt, tokenizer, model, device)

    # Check if the generated text is a valid completion for the prompt
    if check_if_generated_text_is_complete(generated_text):
        prompt_complete = True

    # Update the prompt for the next round of generation
    prompt = generated_text

In [ ]:
generated_text


"What is the number one product ever created and why?\n\nI think the answer is simple. It's a product that is so simple that it's almost impossible to explain.\n, the first product I created, was a simple calculator."

In [ ]:
import re
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def check_if_generated_text_is_complete(generated_text):
    # Check if the generated text ends with a period or question mark
    if not re.search(r'\.\s*$|\?\s*$', generated_text):
        return False

    # Check if the generated text is too short (less than 10 words)
    if len(re.findall(r'\b\w+\b', generated_text)) < 10:
        return False

    # If both checks pass, the generated text is considered a valid completion
    return True
def generate_text(prompt, tokenizer, model, device):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    model.eval()
    with torch.no_grad():
        output = model.generate(inputs,max_length=49, do_sample=False, # Use greedy decoding
                        no_repeat_ngram_size=2, # No repetition penalty
                                num_beams=2,
                        early_stopping=True, # Stop as soon as a period is generated
                        pad_token_id=model.config.eos_token_id)

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output
user_input = request.form["user_input"]
prompt=user_input
# ... tokenizer, model, and device setup ...

# Check if the prompt has already been generated
prompt_complete = False
while not prompt_complete:
    # Generate text with the model
    generated_text = generate_text(prompt, tokenizer, model, device)

    # Check if the generated text is a valid completion for the prompt
    if check_if_generated_text_is_complete(generated_text):
        prompt_complete = True

    # Update the prompt for the next round of generation
    prompt = generated_text